In [26]:
import os
import numpy as np

import tensorflow.compat.v1 as tf

from hybrid_transformer.evaluations.evaluator import Evaluator

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

%load_ext autoreload
%autoreload 2

config = tf.ConfigProto(
    allow_soft_placement=True  # allows DecodeJpeg to run on CPU in Inception graph
)
config.gpu_options.allow_growth = False


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
config

gpu_options {
  allow_growth: true
}
allow_soft_placement: true

In [27]:
evaluator = Evaluator(tf.Session(config=config))

2023-12-07 19:36:44.627347: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 19:36:44.627784: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 19:36:44.628126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [19]:
%cd hybrid-transformer

/raid/aizd/hybrid-transformer


In [20]:
import pynvml

def get_free_memory(gpu_index):
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(int(gpu_index))
    mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    print('Memory Usage:')
    print(f"Free: {mem_info.free // 1024 ** 3 } gb")
    return None

get_free_memory(3)

Memory Usage:
Free: 9 gb


### Obliczanie aktywacji referencyjnych bezpośrednio z obrazków:

In [28]:
cifar_act = evaluator.read_activations("hybrid_transformer/evaluations/openai_eval/cifar10/cifar_10_images.npz")

  0%|                                                                                                                           | 0/6250 [00:00<?, ?it/s]2023-12-07 19:36:54.140829: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:746] failed to allocate 33.62MiB (35258368 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-12-07 19:36:54.141514: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:746] failed to allocate 30.26MiB (31732736 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-12-07 19:36:54.361365: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:746] failed to allocate 6.39MiB (6698752 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-12-07 19:36:54.362129: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:746] failed to allocate 6.39MiB (6698752 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-12-07 19:37:04.363073: I tensorflow/compiler/xla/stream_executor/cuda/cuda

ResourceExhaustedError: Graph execution error:

Detected at node '140039600_384308438/mixed_5/tower/conv_2/conv2d_params' defined at (most recent call last):
Node: '140039600_384308438/mixed_5/tower/conv_2/conv2d_params'
OOM when allocating tensor of shape [7,1,160,192] and type float
	 [[{{node 140039600_384308438/mixed_5/tower/conv_2/conv2d_params}}]]

Original stack trace for '140039600_384308438/mixed_5/tower/conv_2/conv2d_params':


### Wczytywanie już przeliczonych aktywacji

In [29]:
cifar_act = np.load("hybrid_transformer/evaluations/openai_eval/cifar10/cifar_act_0.npz")['arr_0'], np.load("hybrid_transformer/evaluations/openai_eval/cifar10/cifar_act_1.npz")['arr_0']

In [31]:
ref_stats, ref_stats_spatial = evaluator.read_statistics("hybrid_transformer/evaluations/openai_eval/cifar10/cifar_10_images.npz", cifar_act)

### Ścieżka do sampli

In [ ]:
gen_path = "results/samples_model50000.npz"

In [ ]:
gen_act = evaluator.read_activations(gen_path)

In [ ]:
sample_stats, sample_stats_spatial = evaluator.read_statistics(gen_path, gen_act)

In [ ]:
print("Inception Score:", evaluator.compute_inception_score(gen_act[0]))
print("FID:", sample_stats.frechet_distance(ref_stats))
print("sFID:", sample_stats_spatial.frechet_distance(ref_stats_spatial))
prec, recall = evaluator.compute_prec_recall(cifar_act[0], gen_act[0])
print("Precision:", prec)
print("Recall:", recall)

In [35]:
arr = np.load("hybrid_transformer/evaluations/openai_eval/cifar10/cifar_act_0.npz")['arr_0']

In [40]:
arr

array([[2.1053073e-01, 1.5319686e-01, 3.9881843e-01, ..., 4.0128839e-01,
        2.0353584e-01, 3.7829801e-01],
       [9.8507339e-04, 6.7003423e-01, 3.3620864e-01, ..., 4.7808197e-01,
        3.2776386e-01, 1.7593683e+00],
       [1.1272302e-02, 1.5619931e-01, 2.3901385e-01, ..., 9.0339112e-01,
        2.0107277e-01, 4.6611536e-01],
       ...,
       [7.0123874e-02, 9.8172151e-02, 4.9429029e-01, ..., 5.8184545e-02,
        8.4632248e-01, 6.9143265e-01],
       [7.8763766e-03, 3.0279800e-01, 4.6096414e-01, ..., 1.6289216e-02,
        6.4681119e-01, 6.0700029e-01],
       [2.5535578e-01, 3.8893071e-01, 1.7253067e-01, ..., 4.4582954e-01,
        9.2903458e-02, 8.0221958e-02]], dtype=float32)

In [43]:
arr = np.load("hybrid_transformer/evaluations/openai_eval/cifar10/cifar_10_images.npz")['arr_0']

In [45]:
arr.shape

(50000, 32, 32, 3)

In [46]:
arr = np.load("sampled.npz")['arr_0']

In [47]:
arr.shape

(8031, 3, 32, 32)